# Yelp Dataset: Data Cleaning

Limiting the Yelp dataset to **Repair Services** categories only.
Save data into CSV for faster loading

In [1]:
# data handling/prediction
import pandas as pd
import numpy as np

In [2]:
# load in business data
business = pd.read_json('data/business.json', lines=True)
print(business.shape)
business.head()

(192609, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [3]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192609 entries, 0 to 192608
Data columns (total 14 columns):
business_id     192609 non-null object
name            192609 non-null object
address         192609 non-null object
city            192609 non-null object
state           192609 non-null object
postal_code     192609 non-null object
latitude        192609 non-null float64
longitude       192609 non-null float64
stars           192609 non-null float64
review_count    192609 non-null int64
is_open         192609 non-null int64
attributes      163773 non-null object
categories      192127 non-null object
hours           147779 non-null object
dtypes: float64(3), int64(2), object(9)
memory usage: 20.6+ MB


In [4]:
business.is_open.value_counts()

1    158525
0     34084
Name: is_open, dtype: int64

In [5]:
business.categories.notnull().value_counts()

True     192127
False       482
Name: categories, dtype: int64

In [6]:
# filter out 1) closed businesses and 2) entries with missing categories
open_business = business[(business['is_open']==1) & (business.categories.notnull())][['business_id','name','stars','categories']]
open_business = open_business.rename(columns={'name': 'business_name', 'stars': 'business_stars'})
open_business.head()

,business_id,business_name,business_stars,categories
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,2.5,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,4.0,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,5.0,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,4.0,"Plumbing, Shopping, Local Services, Home Servi..."
5,68dUKd8_8liJ7in4aWOSEA,The UPS Store,2.5,"Shipping Centers, Couriers & Delivery Services..."


In [7]:
# limit data to service categories only
services = open_business[open_business.categories.str.contains('Repair', case=False, na=False)]
services.head()

,business_id,business_name,business_stars,categories
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,4.0,"Plumbing, Shopping, Local Services, Home Servi..."
10,fcXOEZdXYeZqnQ3lGlOXmg,Nucleus Information Service,2.0,"Local Services, Professional Services, Compute..."
28,iojTeSaoPuxm4WeCzDUA6w,AW Collision,4.5,"Car Rental, Windshield Installation & Repair, ..."
31,mNBp4KI2goFJKDB9VLGP9w,JSE Automotive Services,5.0,"Automotive, Auto Repair"
87,nkoie_y3fbaQGjZqFGqsrg,Complete Auto Glass,5.0,"Home Services, Auto Glass Services, Windows In..."


In [8]:
# expand out categories list into columns using get_dummies
category_df = services.categories.str.get_dummies(sep = ", ")
print(category_df.shape)
category_df.head()

,ATV Rentals/Tours,Accessories,Accountants,Active Life,Adult Education,Adult Entertainment,Advertising,Air Duct Cleaning,Aircraft Dealers,Aircraft Repairs,...,Well Drilling,Wheel & Rim Repair,Wholesale Stores,Wholesalers,Window Washing,Windows Installation,Windshield Installation & Repair,Wine & Spirits,Women's Clothing,Yelp Events
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [30]:
# remove irrelevant columns (ie. does not contain "Repair")
repair_df = category_df.filter(regex='Repair')
print(repair_df.shape)

(10191, 31)


In [32]:
print("Original Column Names: ")
print(repair_df.columns.values)

Original Column Names: 
['Aircraft Repairs' 'Appliances & Repair' 'Auto Repair' 'Bike Repair'
 'Bike Repair/Maintenance' 'Boat Repair' 'Clock Repair'
 'Commercial Truck Repair' 'Drywall Installation & Repair'
 'Electronics Repair' 'Farm Equipment Repair' 'Foundation Repair'
 'Furniture Repair' 'Generator Installation/Repair' 'Hybrid Car Repair'
 'IT Services & Computer Repair' 'Jewelry Repair' 'Mobile Dent Repair'
 'Mobile Home Repair' 'Mobile Phone Repair' 'Motorcycle Repair'
 'Motorsport Vehicle Repairs' 'RV Repair' 'Sauna Installation & Repair'
 'Shoe Repair' 'Trailer Repair' 'Transmission Repair' 'Watch Repair'
 'Water Heater Installation/Repair' 'Wheel & Rim Repair'
 'Windshield Installation & Repair']


In [33]:
# clean up column names
repair_df.columns = repair_df.columns.map(lambda x: x.replace("/"," and "))
repair_df.columns = repair_df.columns.map(lambda x: x.replace("&","and"))
repair_df.columns = repair_df.columns.map(lambda x: x.replace(" ","_"))
repair_df.columns = repair_df.columns.map(lambda x: x.lower())
print("Revised Column Names: ")
print(repair_df.columns.values)
repair_df.head()

Revised Column Names: 
['aircraft_repairs' 'appliances_and_repair' 'auto_repair' 'bike_repair'
 'bike_repair_and_maintenance' 'boat_repair' 'clock_repair'
 'commercial_truck_repair' 'drywall_installation_and_repair'
 'electronics_repair' 'farm_equipment_repair' 'foundation_repair'
 'furniture_repair' 'generator_installation_and_repair'
 'hybrid_car_repair' 'it_services_and_computer_repair' 'jewelry_repair'
 'mobile_dent_repair' 'mobile_home_repair' 'mobile_phone_repair'
 'motorcycle_repair' 'motorsport_vehicle_repairs' 'rv_repair'
 'sauna_installation_and_repair' 'shoe_repair' 'trailer_repair'
 'transmission_repair' 'watch_repair'
 'water_heater_installation_and_repair' 'wheel_and_rim_repair'
 'windshield_installation_and_repair']


,aircraft_repairs,appliances_and_repair,auto_repair,bike_repair,bike_repair_and_maintenance,boat_repair,clock_repair,commercial_truck_repair,drywall_installation_and_repair,electronics_repair,...,motorsport_vehicle_repairs,rv_repair,sauna_installation_and_repair,shoe_repair,trailer_repair,transmission_repair,watch_repair,water_heater_installation_and_repair,wheel_and_rim_repair,windshield_installation_and_repair
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
31,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [34]:
# merge expanded category list back to original dataframe
services_categories = services.merge(repair_df,left_index=True,right_index=True)
services_categories.head()

,business_id,business_name,business_stars,categories,aircraft_repairs,appliances_and_repair,auto_repair,bike_repair,bike_repair_and_maintenance,boat_repair,...,motorsport_vehicle_repairs,rv_repair,sauna_installation_and_repair,shoe_repair,trailer_repair,transmission_repair,watch_repair,water_heater_installation_and_repair,wheel_and_rim_repair,windshield_installation_and_repair
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,4.0,"Plumbing, Shopping, Local Services, Home Servi...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10,fcXOEZdXYeZqnQ3lGlOXmg,Nucleus Information Service,2.0,"Local Services, Professional Services, Compute...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,iojTeSaoPuxm4WeCzDUA6w,AW Collision,4.5,"Car Rental, Windshield Installation & Repair, ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
31,mNBp4KI2goFJKDB9VLGP9w,JSE Automotive Services,5.0,"Automotive, Auto Repair",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,nkoie_y3fbaQGjZqFGqsrg,Complete Auto Glass,5.0,"Home Services, Auto Glass Services, Windows In...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [35]:
# load in review data (in chunks due to large file size)
reviews = pd.read_json('data/review.json', lines=True, chunksize=10000,
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int})

In [36]:
# iterate through each review chunk and append data 
full_data = []

for review in reviews:
    review = review.drop(['review_id','useful','funny','cool'], axis=1)
    review['date'] = pd.to_datetime(review['date'])
    review_business_merged = review.merge(services_categories, on='business_id', how='inner')
    full_data.append(review_business_merged)

In [37]:
# convert into dataframe
df = pd.concat(full_data, ignore_index=True)
print(df.shape)
df.head()

(226830, 39)


,user_id,business_id,stars,text,date,business_name,business_stars,categories,aircraft_repairs,appliances_and_repair,...,motorsport_vehicle_repairs,rv_repair,sauna_installation_and_repair,shoe_repair,trailer_repair,transmission_repair,watch_repair,water_heater_installation_and_repair,wheel_and_rim_repair,windshield_installation_and_repair
0,vXZeHaLvdfCPATz5BDavow,FQ1wBQb3aNeRMThSQEV0Qg,5,PlumbSmart provided superior service from begi...,2013-12-07 02:32:45,PlumbSmart Plumbing Heating and Air,4.0,"Damage Restoration, Heating & Air Conditioning...",0,0,...,0,0,0,0,0,0,0,1,0,0
1,k5y1Gs82edz90m_MhxPF7Q,FQ1wBQb3aNeRMThSQEV0Qg,5,I could not have asked for a better plumbing e...,2016-07-14 20:00:32,PlumbSmart Plumbing Heating and Air,4.0,"Damage Restoration, Heating & Air Conditioning...",0,0,...,0,0,0,0,0,0,0,1,0,0
2,vDr8hBNYigiY70fk1kF5iQ,hd5xm20tfSa70-6UqD9-bg,1,"Unfortunately, I must recommend not to conduct...",2014-08-10 22:07:35,Fast Fix Jewelry and Watch Repairs,4.5,"Jewelry Repair, Watch Repair, Jewelry, Local S...",0,0,...,0,0,0,0,0,0,1,0,0,0
3,vcdMd3zawCdPjqZ12jUn5w,hd5xm20tfSa70-6UqD9-bg,5,Fast-Fix of Scottsdale is the place to go for ...,2018-04-27 18:36:30,Fast Fix Jewelry and Watch Repairs,4.5,"Jewelry Repair, Watch Repair, Jewelry, Local S...",0,0,...,0,0,0,0,0,0,1,0,0,0
4,Xk5WsRl9fp1U6KzLdWiDVw,hd5xm20tfSa70-6UqD9-bg,1,I had a watch repaired here. When they gave me...,2016-07-03 17:46:53,Fast Fix Jewelry and Watch Repairs,4.5,"Jewelry Repair, Watch Repair, Jewelry, Local S...",0,0,...,0,0,0,0,0,0,1,0,0,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226830 entries, 0 to 226829
Data columns (total 39 columns):
user_id                                 226830 non-null object
business_id                             226830 non-null object
stars                                   226830 non-null int64
text                                    226830 non-null object
date                                    226830 non-null datetime64[ns]
business_name                           226830 non-null object
business_stars                          226830 non-null float64
categories                              226830 non-null object
aircraft_repairs                        226830 non-null int64
appliances_and_repair                   226830 non-null int64
auto_repair                             226830 non-null int64
bike_repair                             226830 non-null int64
bike_repair_and_maintenance             226830 non-null int64
boat_repair                             226830 non-null int64
clock

In [39]:
# save df to csv
df.to_csv('data/services_reviews.csv', index=False)

See **Analysis** notebook for next steps.